# Top Three Salaries in Each Department

**QUESTION**

Find employees who earn the top three salaries in each of the department.

EXAMPLE:

Employee table:

    +----+-------+--------+--------------+
    | Id | Name  | Salary | DepartmentId |
    +----+-------+--------+--------------+
    | 1  | Joe   | 85000  | 1            |
    | 2  | Henry | 80000  | 2            |
    | 3  | Sam   | 60000  | 2            |
    | 4  | Max   | 90000  | 1            |
    | 5  | Janet | 69000  | 1            |
    | 6  | Randy | 85000  | 1            |
    | 7  | Will  | 70000  | 1            |
    +----+-------+--------+--------------+

Department table:

    +----+----------+
    | Id | Name     |
    +----+----------+
    | 1  | IT       |
    | 2  | Sales    |
    +----+----------+

Expected result:

    +------------+----------+--------+
    | Department | Employee | Salary |
    +------------+----------+--------+
    | IT         | Max      | 90000  |
    | IT         | Randy    | 85000  |
    | IT         | Joe      | 85000  |
    | IT         | Will     | 70000  |
    | Sales      | Henry    | 80000  |
    | Sales      | Sam      | 60000  |
    +------------+----------+--------+
    
**TECHNIQUES:**

- SQL:
  - [DENSE_RANK](https://www.mysqltutorial.org/mysql-window-functions/mysql-dense_rank-function/),
    [OVER](https://dev.mysql.com/doc/refman/8.0/en/window-functions-usage.html).
- Pandas:
  - groupby, 
    [rank](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rank.html), 
    sort_values, merge, rename.

**REFERENCE:**
- https://dev.mysql.com/doc/refman/8.0/en/window-functions-usage.html



## Prepare the test data

- Need to install `sqlalchemy` and appropriate drivers (e.g. `mysqlclient` for MySQL).
- Also need to dfine the connection URL.

In [1]:
import pandas as pd
import numpy as np
import os
import sqlalchemy as db

# Create Database Connection.
# Connection info is defined in an environment variable.
CONN_URL = os.environ['TEST_URL']
engine = db.create_engine(CONN_URL)

In [2]:
# Populate the test data
def populate_data(engine, table, data, path):
    """Upload the data to Database and write it to a TSV file."""
    df_tmp = pd.DataFrame(data)
    # Save it to a table
    if engine:
        print("> Uploading data to {} table...".format(table))
        df_tmp.to_sql(table, con=engine, index=False, if_exists='replace')
    # Save it to a file
    if path:
        print("> Saving data to {}...".format(path))
        df_tmp.to_csv(path, index=False, sep="\t")

# Employee
employee_data = {"Id": range(1,8), 
                 "Name": ['Joe', 'Henry', 'Sam', 'Max', 'Janet', 'Randy', "Will"],
                 "Salary": [85000, 80000, 60000, 90000, 69000, 85000, 70000],
                 "DepartmentId": [1, 2, 2, 1, 1, 1, 1]
                }

employee_path = "/tmp/employee.tsv"
populate_data(engine, "Employee", employee_data, employee_path)

# Department
dept_data = {"Id": [1, 2], "Name": ['IT', 'Sales']}
dept_path = "/tmp/orders.tsv"
populate_data(engine, "Department", dept_data, dept_path)


> Uploading data to Employee table...
> Saving data to /tmp/employee.tsv...
> Uploading data to Department table...
> Saving data to /tmp/orders.tsv...


## Pandas Solutions

### Read Data

In [3]:
# Read data from database tables
df_emp = pd.read_sql("SELECT * FROM Employee", engine)
df_dept = pd.read_sql("SELECT * FROM Department", engine)

display(df_emp.head(10))
display(df_dept.head())

,Id,Name,Salary,DepartmentId
0,1,Joe,85000,1
1,2,Henry,80000,2
2,3,Sam,60000,2
3,4,Max,90000,1
4,5,Janet,69000,1
5,6,Randy,85000,1
6,7,Will,70000,1


,Id,Name
0,1,IT
1,2,Sales


In [4]:
# Or, read them from TSV files
df_emp = pd.read_csv(employee_path, sep="\t")
df_dept = pd.read_csv(dept_path, sep="\t")

display(df_emp.head(10))
display(df_dept.head())

,Id,Name,Salary,DepartmentId
0,1,Joe,85000,1
1,2,Henry,80000,2
2,3,Sam,60000,2
3,4,Max,90000,1
4,5,Janet,69000,1
5,6,Randy,85000,1
6,7,Will,70000,1


,Id,Name
0,1,IT
1,2,Sales


### Rank & Merge

- Join the Employee and Department data.
- Then use the **rank()** function on the DataFrame groupby `DepartmentId`

In [5]:
# Rename columns
df_emp2 = df_emp.rename(columns={'Name':'Employee'})
df_dept2 = df_dept.rename(columns={'Id':'DepartmentId', 'Name':'Department'})

# Merge two dataframes on DepartmentId
df_m = df_emp2.merge(df_dept2, how='left', on='DepartmentId')
display(df_m.head())

,Id,Employee,Salary,DepartmentId,Department
0,1,Joe,85000,1,IT
1,2,Henry,80000,2,Sales
2,3,Sam,60000,2,Sales
3,4,Max,90000,1,IT
4,5,Janet,69000,1,IT


In [6]:
# Assign salary rank within each department
df_m['Rank'] = df_m.groupby(['DepartmentId'])['Salary']\
    .rank(method='dense', ascending=False).astype(int)

# Select top 3 salaries and select output columns
df_m[df_m['Rank'] <= 3][['Department', 'Employee', 'Salary']]\
    .sort_values(by=['Department', 'Salary'], ascending=[True, False])

,Department,Employee,Salary
3,IT,Max,90000
0,IT,Joe,85000
5,IT,Randy,85000
6,IT,Will,70000
1,Sales,Henry,80000
2,Sales,Sam,60000


## SQL Solutions

### Salary Comparison
- Join with Department table to get the department name.
- Also only compares with salaries in the same department.

In [7]:
SQL = """
SELECT d.Name as 'Department', e1.Name as 'Employee', e1.Salary as 'Salary'
    FROM Employee e1
    JOIN Department d
        ON e1.DepartmentId = d.Id
    WHERE
        (SELECT COUNT(DISTINCT e2.Salary)
            FROM Employee e2
            WHERE e2.DepartmentId = e1.DepartmentId 
                AND e2.Salary > e1.Salary) < 3
    ORDER BY Department ASC, Salary DESC
"""
pd.read_sql(SQL, engine)

,Department,Employee,Salary
0,IT,Max,90000
1,IT,Joe,85000
2,IT,Randy,85000
3,IT,Will,70000
4,Sales,Henry,80000
5,Sales,Sam,60000


### Dense  Rank Over

- Use **DENSE_RANK** to get the salary rank in each deparemtn.

In [8]:
SQL = """
SELECT d.Name AS 'Department', e.Name AS 'Employee', e.Salary
    FROM (
        SELECT *,
            DENSE_RANK() OVER(PARTITION BY DepartmentID
                ORDER BY Salary DESC) as 'rank'
        FROM Employee
    ) e
    JOIN Department d ON e.DepartmentId = d.Id
    WHERE e.rank <= 3
    ORDER BY d.Name, e.Salary DESC
"""
pd.read_sql(SQL, engine)

,Department,Employee,Salary
0,IT,Max,90000
1,IT,Joe,85000
2,IT,Randy,85000
3,IT,Will,70000
4,Sales,Henry,80000
5,Sales,Sam,60000
